<mark>am Ende ist die Codezelle für die Überprüfung mit der gewählten Konfiguration<mark>

# Quadratic Assignment Problem (QAP)

## Problembeschreibung

- n Anlagen sollen n Standorten zugeordnet werden
- Matrix F definiert die Warenflüsse (Einheiten, Fahrten, etc) zwischen Anlagen 
- Matrix D definiert die Distanzen zwischen Standorten
- Matrix M definiert die Mindestabstände zwischen Anlagen die eingehalten werden müssen (muss nicht beachtet werden)
- keine der Matrizen muss symmetrisch sein
- Input JSON-Files enthalten n, F (entspricht A im File), D und M
- p: Permutation, die Anlagen Standorte zuweist (Indizes bestimme Anlagen und die Werte den Standort)

Ziel ist es die Permutation zu finden für die folgende Kostenformel (Gesamttransportleistung) minimal wird:
$$ \sum_{i=0}^{n-1} \sum_{j=0}^{n-1} f_{ij} d_{p(i)p(j)}$$

Da die Komplexität n! ist, ist QAP für n > 30 nicht optimal in angemessener Zeit zu lösen (Loiola, E. M., De Abreu, N. M. M., Boaventura-Netto, P. O., Hahn, P., & Querido, T. (2007). A survey for the quadratic assignment problem. European journal of operational research, 176(2), 657-690).
Deshalb müssen Heuristiken und Meta-Heuristiken benutzt werden um möglichst gute Lösungen zu finden.

## Zielfunktionsimplenetierung

Um Lösungen bewerten zu können muss die Formel in eine Funktion gegossen werden, die als Input eine Permutation erhält und dann die Kosten berechnet. Diese wird der Code sein der am häufigsten in der Meta-Heuristik ausgeführt wird und sollte deswegen effizient sein. Zuerst hatte ich eine Python-Implementierung, aber die war mir nicht schnell genug. Numpy ist schneller als Python, weil es die Operationen vektorisiert (quasi parallel) und direkt in C ausführt. Das kommt mit etwas Overhead und war für kleinere n (z.B. n=4) langsamer, aber für die Testinstanzen ist es deutlich schneller. Wenn M berücksichtigt werden soll, wird überprüft, ob die Anlagen den Mindestabstand einhalten und wenn nicht werden die Kosten auf unendlich gesetzt. 

```python
   if method == "numpy":
            D_subset = D[np.ix_(permutation[placed], permutation[placed])]
            if input_data.consider_minimum_distance:
                if np.any(D_subset < M):
                    return np.inf
            cost = np.sum(F * D_subset)
            return cost
    elif method == "python-loop"
        cost = 0
            for facility_i, facility_i in enumerate(permutation):
                for facility_j, location_j in enumerate(permutation):
                    distance = D[location_i, location_j]
                    if input_data.consider_minimum_distance:
                        if distance < M[facility_i, facility_j]:
                            return np.inf
                    cost += F[facility_i, facility_j] * distance
            return cost
```
Code aus [Solution Klasse](Solution.py) (Anmerkung: der echte Code enthält noch die Option, bei Permutationen mit nicht zugewiesenen Werten nur die Kosten der zugewiesenen zu berechnen -> wird für ein Eröffnungsverfahren benötigt).


np.idx_ bekommt als Input zwei Arrays mit Zeilen und Spalten der Matrize in der gewünschten Reihenfolge und daraus wird ein Subset, eine Neuanordnung der Spalten/Zeilen oder beides. Durch die Numpy-Implementierung wird diew Zielevaluierung um circa 10x schneller und dadurch lassen sich deutlich mehr Berechnungen innerhalb 3 Minuten durchführen.

In [4]:
import numpy as np
from timeit import default_timer
from InputData import *
from Solution import *

def analyze_objective_function(path: str, iterations: int, only_numpy: bool = False):
    input_data = InputData(path)
    n = input_data.n
    F = input_data.F
    D = input_data.D

    if only_numpy:
        start = default_timer()
        for i in range(iterations): 
            Solution.calculate_cost(np.array([x for x in range(n)]), input_data)
        end = default_timer()
        time_numpy = end - start
        time_python = 0.0
    else:   
        start = default_timer()
        for i in range(iterations): 
            Solution.calculate_cost(np.array([x for x in range(n)]), input_data)
        end = default_timer()
        time_numpy = end - start
        start = default_timer()
        for i in range(iterations):
            Solution.calculate_cost(np.array([x for x in range(n)]), input_data, "python-loop")
        end = default_timer()
        time_python = end - start



    print(f"n: {n}, time with python loops: {time_python:.2f}s, time with numpy: {time_numpy:.2f}s")


In [5]:
iterations = 100_000
paths = ["scr12.json", "scr15.json", "scr20.json", "tai25b.json"]

print(f"Measures for {iterations} iterations")
for path in paths:
    analyze_objective_function(f"Testinstanzen/{path}", iterations)

Measures for 100000 iterations
n: 12, time with python loops: 7.94s, time with numpy: 3.42s
n: 15, time with python loops: 10.60s, time with numpy: 3.33s
n: 20, time with python loops: 19.32s, time with numpy: 2.92s
n: 25, time with python loops: 27.89s, time with numpy: 3.74s


In [6]:
iterations = 1_000_000
for path in paths:
    analyze_objective_function(f"Testinstanzen/{path}", iterations, only_numpy=True)

n: 12, time with python loops: 0.00s, time with numpy: 31.44s
n: 15, time with python loops: 0.00s, time with numpy: 31.40s
n: 20, time with python loops: 0.00s, time with numpy: 34.22s
n: 25, time with python loops: 0.00s, time with numpy: 38.70s


👉 Ohne die numpy Implementierung würden deutlich weniger Nachbarschaften in Frage kommen, da sie einfach zu groß wären. 

# Variable Neighborhood Search (VNS)

## Grundlagen 
VNS ist eine der beliebtesten Meta-Heuristiken. Ihr Input beinhaltet eine anfängliche Permutation und eine geordnete Menge an unterschiedlichen Nachbarschaften. Hauptsächlich besteht VNS aus 3 Komponenten. Shake, LocalSearch und Neighborhood Change Procedure. Shaking ist der Mechanismus um aus Tälern mit lokalem Optimum herauszukommen. LocalSearch ist die Suche nach einem lokalem Optimum innerhalb eines Tals. Täler können aus mehreren kleineren Tälern bestehen, die wiederum lokale Optima besitzten. Erst, wenn das Ergebniss der lokalen Suche besser ist als die vorherige Lösung, wird die Neue akzeptiert (falsch in "Beleginformationen_PW.pdf"). Die Neighborhood Change Procedure definiert, welche Nachbarschaft als nächstes betrachtet werden soll, wenn eine neue Lösung gefunden wurde. VNS besitzt in der Regel keine numerischen Parameter sondern eine oder zwei Menge(n) an Nachbarschaften, die jeweils beim Shaking und bei der Suche verwendet werden. Bei der einfachsten Form sind diese Mengen identisch.

## Ablauf
Eine initiale Permutation wird mittels einer Heuristik erzeugt und als Start für die VNS genutzt. Die erste Nachbarschaft wird ausgewählt und die Schleife betreten. Bei der ersten Iteration wird Shaking übersprungen und es geht direkt zur Suche. Bei der Suche werden alle benachbarten Permutationen (für die aktuelle Nachbarschaft) überprüft (BestImprovement) und die beste ausgewählt. Dies wird durchgeführt bis sich die Lösung nicht verbessert. Wenn sich die Lösung während der Suche nicht verbessert hat, wird in der Neichborhood Change Procedure die nächste Nachbarschaft ausgewählt. Bei Verbesserung setzt man die aktuelle Nachbarschaft auf die Erste zurück. Es wird im Gegensatz zu anderen Meta-Heuristiken niemals eine schlechtere Lösung akzeptiert. Es können schlechtere Lösungen beim Shaking erzeugt und in der Suche deren Nachbarschaften nach besseren Lösungen überprüft werden. Bei der zweiten Iteration wird Shaking angewandt und es wird zufällig ein Move in der aktuellen Nachbarschaft erzeugt und in der Suche weiter überprüft. Basierend auf dem Suchergebnis wird die nächste Nachbarschaft festgelegt. Diese Iteration der 3 Komponente wird solange gemacht bis ein Abbruchkriterium erreicht ist (Zeitlimit überschritten). Wenn alle Nachbarschaften ohne Verbesserung durchlaufen wurden, wird wieder bei der ersten Angefangen. 
Es lohnt sich Nachbarschaften nach aufsteigender Komplexität zu ordnen und genestete Nachbarschaften zu wählen. Ersteres wird gemacht, weil kleinere Nachbarschaften schneller durchsucht werden können und größere Nachbarschaften eigenen sich um aus dem Tal mit lokalem Optimum hinauszuspringen. Zweiteres ist hilfreich, weil die kleineren Nachbarschaften auf diese Weise gründlicher durchsucht werden.

![Grundlegender Ablauf von VNS](Content/VNS.png)

## Variationen
Es gibt einige Varianten von VNS. Die oben beschriebene ist Base VNS. Bei den anderen Varianten gibt es entweder Abweichungne zwischen den Nachbarschaftsmengen zwischen Shaking und Search und/oder Variationen der Neighborhood Change Procedure. 
Bei General VNS wird statt der LocalSearch eine Variable Neighborhood Descent durchgeführt. General VND besitzt eine andere Menge von Nachbarschaften für die Search als für das Shaking. In der Suche wird zuerst die lokale Suche für die erste Nachbarschaft ausgeführt. Wenn dies zu keiner Verbesserung geführt hat, wird die lokale Suche für die Zweite ausgeführt und so weiter, bis eine bessere Lösung als die aktuelle gefunden wurde. Bei diesen Nachbarschaften wird allerdings die Suche aufgehört, wenn für alle keine bessere Lösung gefunden wurde.

Bei den Variationen der Search Procedure gibt es 3 Varianten: 
- Sequential: der oben erläuterte Ablauf, der die Nachbarschaft auf die erste, wenn eine bessere Lösung gefunden wurde, ansonst auf die nächste Nachbarschaft in der Reihenfolge setzt
- Cyclic: alle Nachbarschaften werden einmal durchlaufen und wenn eine bessere
- Pipe: 
Bei allen 3 Varianten wird die Nachbarschaft auf die erste zurückgesetzt, wenn alle ohne Verbesserung durchlaufen wurden.

Bei General VNS bezieht sich die Veränderungen der Nachbarschaften nur auf die für das Shaking. Innerhalb der VND kann es ebenfalls zu von der Shaking Nachbarshaften unabhängigen Veränderungen der Nachbarschaften kommen. Diese können ebenfalls eine der drei Change Procedure sein und es ergeben sich folgende Varianten, die diesmal die Seach Neighborhood verändern: 
- bei B-VND ist sie Sequential: nächste Nachbarschaft auswählen außer es kommt zur Verbesserung (dann erste)
- bei C-VND ist sie Pipe: bei besserer Lösung wird nocheinmal die Suche auf aktueller Nachbarschaft durchgeführt, sonst nächste ausgewählt
- bei P-VND ist sie Cyclic: alle Nachbarschaften werden genau einmal durchlaufen, immer nächste auswählen

Außerdem gibt es noch:
- U-VND: ähnlich zu LocalSearch, allerdings werden alle Nachbarschaften vereinigt und in Suche verwendet
- Reduced VNS: Search-Komponente wird komplett übersprungen (nur durch Shaking wird die Lösung eventuell verbessert). 
- Skewed VNS: durch Parameter Alpha können schlechtere Lösungen in Neighborhood Change angenommen werden, hilft bei Exploration von anderen weit entfernten Tälern

## Quellen
- Mladenović, N., & Hansen, P. (1997). Variable neighborhood search. Computers & operations research, 24(11), 1097-1100.
- Hansen, P., & Mladenovic, N. (2003). A tutorial on variable neighborhood search. Les Cahiers du GERAD ISSN, 711, 2440.- Hansen, P., Mladenović, N., Todosijević, R., & Hanafi, S. (2017). Variable neighborhood search: basics and variants. EURO Journal on Computational Optimization, 5(3), 423-454.

# Architektur

![Klassendiagramm](Content/classes.png)

Der [Solver](Solver.py) ist die Schnittstelle zum User. Es ist die einzige Klasse, die vom Client instanziiert werden muss. Sie erstellt [InputData](InputData.py)-Objekt, einen [Logger](Logger.py) und einen [Timer](Timer.py). Diese drei Objekte werden ebenfalls in den anderen Klassen benötigt, damit jedes Objekt Logs schreiben, die Inputs lesen und die Zeit im Blick haben kann. In InputData werden alle wichtigen Inputs von einer angegebenen JSON eingelesen. Das Logger-Objekt dient zum Debugging, zum Persistieren der Logs und zum ausführlicheren Loggen zur Laufzeit. Der Timer ist die globale Stoppuhr des Solvers. Der Solver als ganzes soll nicht das Zeitlimit überschreiten. Er enthält alle Utility-Funktionen zum Thema Zeit. 

Die Zeitmessung wird erst gestartet, wenn die run-Methode des Solvers ausgeführt wird. Der Solver instanziiert Objekte von [ConstructiveHeuristic](ConstructiveHeuristic.py) und [VariableNeighborhoodSearch](VariableNeighborhoodSearch.py), initialisiert diese mit den Nutzer-definierten Parametern und führt sie aus. Er exposed an ConstructiveHeuristic und VariableNeighborhoodSearch eine *update* Funktion, die die aktuelle Solution des Solvers verwendet, wenn die neue besser ist. Außerdem besitzt er noch eine Methode um die beste gefundene Solution und die Logs in entsprechenden Ordnern zu speichern.

Eine Permutation ist Teil von einer [Solution](Solution.py). Die Gesamttransportleistung wird bei der Initialisierung ausgehend von der Permutation berechnet. Zusätzlich gibt es eine Funktion, die das Objekt mit einer vorherigen Lösung vergleicht (Möglichkeit ein Alpha zu definieren für Skewed VNS) und eine, die das Objekt in Relation zur optimalen Lösung setzt. In einer Solution kann ebenfalls die Nachbarschaft fürs Shaking und Searching für die spätere Analyse getrackt werden

In ConstructiveHeuristics existieren die verschiedenen Eröffnungsverfahren in ihren eigenen Methoden, die durch die run-Methode ausgefwählt und ausgeführt werden und die Lösung des Solvers wird geupdatet. Die VNS-Klasse besitzt ebenfalls eine *run* Methode, die die Koordination übernimmt. In ihr werden die verschiedenen gewünschten [Neighborhoods](Neighborhood.py) für VNS und VND erzeugt und getrackt, welche Nachbarschaft in den beiden Routinen gerade dran ist. Dafür mitverantwortlich ist die *neichborhood_change* Methode. Sie und die Helper, die sie aufruft, werden für VNS und VND verwendet. 

Die Base Class Neighborhood besitzt 5 Kinder: *K_SwapNeighborhood*,*K_InsertionNeighborhood*, *K_CutNeighborhood*, *K_PermuteNeighborhood* und *K_BlockNeighborhood*. Sie implementiert das Shaken, die lokale Suche innerhalb einer Nachbarschaft. Letztere evaluiert so lange die Moves (FirstImprovement oder BestImprovement), bis keine neue Solution gefunden wurde. Jedes Nachbarschaft-Kind muss  zwei Methoden implementieren: *_get_all_combinations* (erzeugt alle Moves innerhalb einer Neighborhood) und *_make_move* (verändert die Permutation und liefert ein neues Solution-Objekt zurück). 

In der VNS-Klasse werden die *shake* und *local_search* Funktionen durch die dazugehörigen Wrapper funktionen aufgerufen für die ausgewählte Nachbarschaft. In der *local_search_procedure* ist außerdem die ganze VND-Logik zu finden und Reduced VNS. Außerdem gibt es noch ein [helper-File](helper.py) mit der Definition der Hamming-Distanz für Skewed VNS.

[Detailiertes Klassendiagramm mit Attributen und Methoden](Content/classes-detailed.png)

# Constructive Heuristics

## Beschreibung der Heuristiken

Ich habe 4 unterschiedliche Eröffnungsverfahren implementiert.
Die letzten drei stammen aus **Müller-Merbach, H. (1979). Optimale Reihenfolgen (Vol. 15). Springer-Verlag**
Alle können Lösungen mit unendlichen Kosten unter Berücksichtigung von Mindestabständen erzeugen.
Random und increasing_dof haben die Option den Vorgang öfter zu wiederholen und die beste Lösung zurückzugeben.

### Random
- zufällige Auswahl von n-Zahlen ohne Zurücklegen  

### Ordered Greedy
- Anlagen und Standorte werden nach abgesteigenden Flüssen bzw. aufsteigen Distanzen geordnet 
- dann wird dem Standort mit der kleinsten Distanz die Anlage mit den meisten Zu- und Abflüssen zugeordnet
- Zuweisung von Standort mit zweit-kleinster Distanz zu allen Anlagen und Anlage mit zweit-kleinsten Flüssen
- Iteration bis alle zugewiesen

### Dynamic Ordered Greedy
- erste Zuordnung ist die gleiche wie bei Ordered Greedy
- danach werden die Flüsse und Distanzen zwischen zugewiesenen Standort/Anlagen und nicht zugewiesenen berechnet
- nicht zugewiesener Standort mit der kleinsten Distanz zu den zugewiesenen Standorten bekommt die Anlage mit den größten Flüssen zu den nicht zugewiesenen Anlagen
- wiederholen bis alle Standorte/Anlagen zugewiesen

### Increasing Degree of Freedom:

- bei den anderen Greedy Algorithmen sinken die Möglichkeiten nach jeder Zuordnung, bei dieser Steigen sie erst und sinken erst Richtung Ende
- Input ist eine Reihenfolge mit der Anlagen den Algorithmus durchlaufen (kann als Parameter gesetzt werden, in meiner Implementation wird sie zufällig generiert)
- erste Anlage wird dem ersten Standort zugewiesen (könnte auch random sein), da noch keine Kosten für diese Zuweisung ermittelt werden können, weil es keine Distanzen/Flüsse zu anderen Standorten/Anlagen gibt
- es gibt zwei Arten von Moves: die zweite Anlage kann jetzt entweder einem freien Standort zugewiesen werden (place) oder sie wird einem besetztem Standort zugewiesen und die verdrängte Anlage muss sich einen freien Platz suchen (swap)
- für jede Möglichkeit werden die Kosten berechnet und dann die mit den wenigsten ausgewählt
- eigene Ergänzung: wenn Mindestdistanzen berücksichtigt werden, ordnet die Maschinen absteigend nach der Summe ihrer Mindestdistanzen (hilft beim finden von nicht-inf Lösungen, da die direkt am Anfang platziert werden und so nur umgerückt werden können, wenn die Lösung besser wird und nicht inf)


## Analyse der Constructive Heurisistcs

In [14]:
from Solver import *
import pandas as pd

def run_constructive_heuristic(inputData):
    rng = np.random.default_rng(4)
    timer = Timer(5)
    logger = Logger()

    constructive_heuristic = ConstructiveHeuristic(inputData, rng, timer, logger)

    df = pd.DataFrame(columns=["Algorithm", "Cost", "Diff %", "Time (s)"])

    timer.start_timer()
    # set seed back for comparability
    constructive_heuristic.rng = np.random.default_rng(4)
    solution_dof_50 = constructive_heuristic.increasing_dof(50)
    diff = solution_dof_50.check_optimal_solution(inputData, print_diff=False)
    df.loc[len(df)] = ["Increasing Degree of Freedom 50x", 
                       solution_dof_50.cost,
                       round(diff, 2),
                       round(timer.get_time(), 4)]

    timer.start_timer() 
    constructive_heuristic.rng = np.random.default_rng(4)
    solution_dof_10 = constructive_heuristic.increasing_dof(10)
    diff = solution_dof_10.check_optimal_solution(inputData, print_diff=False)
    df.loc[len(df)] = ["Increasing Degree of Freedom 10x",
                       solution_dof_10.cost,
                       round(diff, 2),
                       round(timer.get_time(), 4)]
    
    timer.start_timer() 
    constructive_heuristic.rng = np.random.default_rng(4)
    solution_dof_5 = constructive_heuristic.increasing_dof(5)
    diff = solution_dof_5.check_optimal_solution(inputData, print_diff=False)
    df.loc[len(df)] = ["Increasing Degree of Freedom 5x",
                       solution_dof_5.cost,
                       round(diff, 2),
                       round(timer.get_time(), 4)]

    timer.start_timer()
    constructive_heuristic.rng = np.random.default_rng(4)   
    solution_dof_1 = constructive_heuristic.increasing_dof(1)
    diff = solution_dof_1.check_optimal_solution(inputData, print_diff=False)
    df.loc[len(df)] = ["Increasing Degree of Freedom 1x",
                       solution_dof_1.cost,
                       round(diff, 2),
                       round(timer.get_time(), 4)]

    timer.start_timer()
    constructive_heuristic.rng = np.random.default_rng(4)
    solution_random_10000 = constructive_heuristic.random(10000)
    diff = solution_random_10000.check_optimal_solution(inputData, print_diff=False)
    df.loc[len(df)] = ["Random 10000x",
                       solution_random_10000.cost,
                       round(diff, 2),
                       round(timer.get_time(), 4)]

    timer.start_timer()
    constructive_heuristic.rng = np.random.default_rng(4)
    solution_random_1 = constructive_heuristic.random(1)
    diff = solution_random_1.check_optimal_solution(inputData, print_diff=False)
    df.loc[len(df)] = ["Random 1x",
                       solution_random_1.cost,
                       round(diff, 2),
                       round(timer.get_time(), 4)]

    timer.start_timer()
    solution_ordered_greedy = constructive_heuristic.ordered_greedy()
    diff = solution_ordered_greedy.check_optimal_solution(inputData, print_diff=False)
    df.loc[len(df)] = ["Ordered Greedy",
                       solution_ordered_greedy.cost,
                       round(diff, 2),
                       round(timer.get_time(), 4)]

    timer.start_timer()
    solution_dynamic_ordered_greedy = constructive_heuristic.dynamic_ordered_greedy()
    diff = solution_dynamic_ordered_greedy.check_optimal_solution(inputData, print_diff=False)
    df.loc[len(df)] = ["Dynamic Ordered Greedy",
                       solution_dynamic_ordered_greedy.cost,
                       round(diff, 2),
                       round(timer.get_time(), 4)]
    m = "with" if inputData.consider_minimum_distance else "without"
    print(f"\nResults for {inputData.name} {m} minimum distance:")
    print(df.to_string(index=False))

In [15]:
from Solver import *

paths = ["scr12.json", "scr15.json", "scr20.json", "tai25b.json"]

for path in paths:
    inputData = InputData(f"Testinstanzen/{path}", consider_minimum_distance=False)
    run_constructive_heuristic(inputData)
    inputData = InputData(f"Testinstanzen/{path}", consider_minimum_distance=True)
    run_constructive_heuristic(inputData)



Results for scr12 without minimum distance:
                       Algorithm  Cost  Diff %  Time (s)
Increasing Degree of Freedom 50x 31884    1.51    0.6126
Increasing Degree of Freedom 10x 32956    4.92    0.1006
 Increasing Degree of Freedom 5x 32956    4.92    0.0425
 Increasing Degree of Freedom 1x 35766   13.87    0.0088
                   Random 10000x 36686   16.80    0.4349
                       Random 1x 56302   79.25    0.0003
                  Ordered Greedy 37236   18.55    0.0003
          Dynamic Ordered Greedy 40018   27.41    0.0014

Results for scr12 with minimum distance:
                       Algorithm    Cost  Diff %  Time (s)
Increasing Degree of Freedom 50x 36072.0    6.11    0.9860
Increasing Degree of Freedom 10x 37714.0   10.94    0.1857
 Increasing Degree of Freedom 5x 37714.0   10.94    0.0705
 Increasing Degree of Freedom 1x 39754.0   16.94    0.0148
                   Random 10000x 37282.0    9.67    1.0570
                       Random 1x     inf     i

👉 Dynamic Ordered nicht immer besser als Ordered Greedy, dauert aber länger

👉 increasing_dof ist sehr gut 10 Ausführungen können unter einer Sekunde (außer tai25b mit M ist ganz knapp drüber) durchgeführt werden und die weiteren Versuche verbessern selten den Wert 

👉 bei tai25b mit M wird mit increasing_dof zufällig eine sehr gute Lösung getroffen (nicht mit anderen seeds kommt man in die Nähe von 2 Prozent)

👉 increasing_dof findet mit meiner heuristischen Ermittlung der Anlagenreihenfolge bei Berücksichtigung von M immer bei Einmaliger Ausführung eine Lösung (war vorher nicht zwangsläufig der Fall). 

👉 beide simplere Greedy Verfahren liefern beide mit der Einschränkung keine valide Lösung

Ich werde eine andere Seed beim Testen wählen (um Overfitting und Glück etwas entgegenzuwirken) und mit 1-increasing_dof starten. Damit der Einfluss nicht zu groß vom Eröffnungsverfahren ist und ich mehr die VNS untersuchen kann.

# Neighborhoods

## Grundlagen

Das wichtigste bei VNS ist eine große Auswahl an möglichen Neighborhoods zu haben. Deshalb habe ich Neighborhoods immer mit einem Parameter k implementiert. Bei den Neighborhoods Swap und Insertion ist k die Anzahl der hintereinanter ausgeführten k=1 Moves. Wie bereits erwähnt sind genestete Nachbarschaften vom Vorteil. Eine (k+1)-Nachbarschaft enthält alle Moves (Moves = Kombinationen) der k-Nachbarschaft.
Die Kombinationen werden bei der initialisierung erzeugt. Dies ist nicht notwendig bei FirstImrovement und man hätte einen Python Generator (yield) benutzen können, allerdings wird auf diese Weise das Shaking viel einfacher (random Integer innerhalb der Kombinationslänge) und bei der Evaluierung muss nur über die Kombinationen iteriert werden.

#### k-Swap
Es werden swaps von 2 Werten hintereinander ausgeführt. Kombinationen von k-Swaps sind nur zulässig, wenn die Indizes der einzelnen Swaps disjunkt sind. Dadurch sind sie nicht genestet, aber wachsen nicht so schnell mit n. 

#### k-Insertion
Es werden mehrere 1-Insertions hintereinander ausgeführt. Die Reihenfolge spielt eine Rolle, wenn die Insertions sich gegenseitig beeinflussen (überlappende Indizes). Die selbe Insertion hintereinander ist nur eine no-op, wenn sie direkt nebeneinander liegen. Außerdem dürfen die Insertions sich nicht gegenseitig aufheben (gilt nur für k=2). 2-Insertion enthält alle Moves von 1-Insertion
K-Insertion Implementierung enthält mehrere Moves, die zur gleichen Permutation führen können (da Reihenfolge beachtet wird und nicht überlappende Indizes entstehen können) und kann zu no-op Moves führen bei k > 2.

#### k-Block
k definiert die Länge der Insertion-Blöcke. 1-Block entspricht 1-Insertion.

#### k-Permute
Eine Kombination entspricht k-Indizes der Ausgangspermutation, die in k!-1 Möglichkeiten neu angeordnet werden können. Dadurch, dass nur die initiale Kombination nicht erlaubt ist, beinhaltet k-Permute alle (k-1)-Permute Moves. 2-Permute entspricht 1-Swap.
Beispiel für Nestung von 2- und 3-Permute: ((1,2,3),(0,2,1)) entspricht ((2,3),(0,1))

#### k-Cut
Inspiriert von k-Opt, aber ohne reversing. Wollte noch eine Nachbarschaft, die mehr Chaos verursacht (gut fürs Shaking) und auch nested ist. Eine Kombination besteht aus Indizes vor denen die Permutation geteilt wird und aus (k+1)!-1 Neuanordnungen der k+1 entstandenen Splits. K-Cut enthält alle (C-1)-Moves. Die Cut-Indizes müssen größer als 0 sein und kleiner als n, damit keine leeren Listen entstehen (es wird immer vor ihnen gecuttet)
Beispiel Nestung von 1- und 2-Cut: ((1,2),(1,2,0)) entspricht ((1),(1,0)).

[0,1,2,3,4] cut bei (1,2) -> [0], [1], [2,3,4] -> ordnen durch (1,2,0) -> [1,2,3,4,0]

[0,1,2,3,4] cut bei (1) -> [0], [1,2,3,4] -> ordnen durch (1,0) -> [1,2,3,4,0]



## Neighborhoods Analyse

In [4]:
from Solver import * 
import pandas as pd

# load file with input data
def analyze_neighborhoods(inputData: InputData, nhood_types: list[str]):
    rng = np.random.default_rng(4)
    logger = Logger()
    timer = Timer(10000)

    timer.start_timer()

    df = pd.DataFrame(columns=['Type', 'Length', 'Time (s)'])

    vns = VariableNeighborhoodSearch(inputData, rng, logger=logger, timer=timer)

    params = {
        "neighborhood_types": nhood_types,
        "neighborhood_evaluation_strategy": "BestImprovement", 
        "search_procedure": "LocalSearch",
        "neighborhood_change_procedure": "Sequential"
    }

    vns.initialize(params)
    vns.initialize_neighborhoods()
    
    for neighborhood in vns.neighborhoods.values():
        df.loc[len(df)] = [neighborhood.type, len(neighborhood.combinations), neighborhood.time_taken]

    df = df.sort_values("Length")
    print("Results for ", inputData.name)
    print(df.to_string(index=False))

ImportError: dlopen(/opt/miniconda3/lib/python3.12/site-packages/numpy/core/_multiarray_umath.cpython-312-darwin.so, 0x0002): Library not loaded: @loader_path/libgfortran.5.dylib
  Referenced from: <7048D1FD-B9A9-3F6A-B752-AE8BE69F6C3F> /opt/miniconda3/lib/python3.12/site-packages/numpy/.dylibs/libopenblas64_.0.dylib
  Reason: tried: '/opt/miniconda3/lib/python3.12/site-packages/numpy/.dylibs/libgfortran.5.dylib' (no such file), '/usr/local/lib/libgfortran.5.dylib' (no such file), '/usr/lib/libgfortran.5.dylib' (no such file, not in dyld cache)

ImportError: dlopen(/opt/miniconda3/lib/python3.12/site-packages/numpy/core/_multiarray_umath.cpython-312-darwin.so, 0x0002): Library not loaded: @loader_path/libgfortran.5.dylib
  Referenced from: <7048D1FD-B9A9-3F6A-B752-AE8BE69F6C3F> /opt/miniconda3/lib/python3.12/site-packages/numpy/.dylibs/libopenblas64_.0.dylib
  Reason: tried: '/opt/miniconda3/lib/python3.12/site-packages/numpy/.dylibs/libgfortran.5.dylib' (no such file), '/usr/local/lib/libgfortran.5.dylib' (no such file), '/usr/lib/libgfortran.5.dylib' (no such file, not in dyld cache)

In [ ]:
paths = ["scr12.json", "scr15.json", "scr20.json", "tai25b.json"]

nhood_types = ["1-Swap", "2-Swap", "3-Swap", "4-Swap", 
                   "1-Insertion", "2-Insertion",
                   "1-Cut","2-Cut", "3-Cut", "4-Cut", "5-Cut",
                   "2-Permute", "3-Permute", "4-Permute", "5-Permute",
                   "1-Block", "2-Block", "3-Block", "4-Block", "5-Block", "6-Block", "7-Block", "8-Block"]

for path in paths:
    inputData = InputData(f"Testinstanzen/{path}", consider_minimum_distance=False)
    # higher swaps and insertions take too long to compute
    analyze_neighborhoods(inputData, nhood_types)

Results for  scr12
       Type  Length  Time (s)
      1-Cut      11  0.000014
    8-Block      20  0.000004
    7-Block      30  0.000007
    6-Block      41  0.000009
    5-Block      53  0.000007
     1-Swap      66  0.000048
  2-Permute      66  0.000029
    4-Block      67  0.000012
    3-Block      83  0.000016
    2-Block     101  0.000017
    1-Block     132  0.000034
1-Insertion     132  0.000104
      2-Cut     275  0.000043
  3-Permute    1100  0.000130
     2-Swap    1485  0.001045
      3-Cut    3795  0.000293
  4-Permute   11385  0.000893
     3-Swap   13860  0.030010
2-Insertion   17270  0.011515
      4-Cut   39270  0.002841
     4-Swap   51975  0.393862
  5-Permute   94248  0.006957
      5-Cut  332178  0.023461
Results for  scr15
       Type  Length  Time (s)
      1-Cut      14  0.000013
    8-Block      56  0.000008
    7-Block      70  0.000012
    6-Block      86  0.000013
    5-Block     104  0.000015
     1-Swap     105  0.000061
  2-Permute     105  0.000040
  

👉 für BestImprovement unter 100k Moves bleiben (100k Zielfunktionsberechnungen dauert um die zwei Sekunden)

👉 für FirstImprovement und bei separaten Shake-Nachbarschaften (General VNS) können auch größere verwendet werden

👉 4-Swap sollte nicht verwendet werdern, weil die Initialisierungszeit zu lange dauert, für alle anderen Nachbarschaften akzeptabel

👉 höhere Swaps, Permutes und Insertion nicht initialisierbar in Zeit- und Memorybeschränkungen

## Neighborhoods Testing/Showcasing

In [2]:
import numpy as np
from Solver import *


rng = np.random.default_rng(4)
logger = Logger()
timer = Timer(100)

timer.start_timer()

input_data = InputData("Testinstanzen/testing.json", consider_minimum_distance=False)

nhood_3swap = K_SwapNeighborhood(input_data, rng, logger, timer, "3-Swap")
nhood_3block = K_BlockNeighborhood(input_data, rng, logger, timer, "3-Block")
nhood_3cut = K_CutNeighborhood(input_data, rng, logger, timer, "3-Cut")
nhood_3permute = K_PermuteNeighborhood(input_data, rng, logger, timer, "3-Permute")
nhood_3insertion = K_InsertionNeighborhood(input_data, rng, logger, timer, "3-Insertion")

permutation = np.array(range(input_data.n))

def get_random_combination(nhood):
    return nhood.combinations[rng.integers(0,len(nhood.combinations))]

combination_3swap = get_random_combination(nhood_3swap)
combination_3block = get_random_combination(nhood_3block)
combination_3cut = get_random_combination(nhood_3cut)
combination_3permute = get_random_combination(nhood_3permute)
combination_3insertion = get_random_combination(nhood_3insertion)


# combination_3swap = 
# combination_3block = (3, 1)
# combination_3cut =
# combination_3permute = 
# combination_3insertion = ((0, 1), (2, 3), (4, 5))

move_3swap = nhood_3swap._make_move(permutation, combination_3swap)
move_3block = nhood_3block._make_move(permutation, combination_3block)
move_3cut = nhood_3cut._make_move(permutation, combination_3cut)
move_3permute = nhood_3permute._make_move(permutation, combination_3permute)
move_3insertion = nhood_3insertion._make_move(permutation, combination_3insertion)

print("3-Swap Neighborhood")
print("-"*100)
print(f"Combinations: {nhood_3swap.combinations}")
print(f"Move: {combination_3swap}")
print("Initial permutation vs. 3-swap permutation:")
print(permutation)
print(move_3swap.permutation)
print("-"*100)
print("-"*100)
print("3-Block Neighborhood")
print("-"*100)
print(f"Combinations: {nhood_3block.combinations}")
print(f"Move: {combination_3block}")       
print("Initial permutation vs. 3-block permutation:")
print(permutation)
print(move_3block.permutation)  
print("-"*100)
print("-"*100)
print("3-Cut Neighborhood")
print("-"*100)
print(f"Combinations: {nhood_3cut.combinations}")
print(f"Move: {combination_3cut}")
print("Initial permutation vs. 3-cut permutation:")
print(permutation)
print(move_3cut.permutation)
print("-"*100)
print("-"*100)
print("3-Permute Neighborhood")
print("-"*100)
print(f"Combinations: {nhood_3permute.combinations}")
print(f"Move: {combination_3permute}")
print("Initial permutation vs. 3-permute permutation:")
print(permutation)
print(move_3permute.permutation)
print("-"*100)
print("-"*100)
print("3-Insertion Neighborhood")
print("-"*100)
print(f"Combinations: {nhood_3insertion.combinations}")
print(f"Move: {combination_3insertion}")
print("Initial permutation vs. 3-insertion permutation:")
print(permutation)
print(move_3insertion.permutation)
print("-"*100)
print("-"*100)


3-Swap Neighborhood
----------------------------------------------------------------------------------------------------
Combinations: [((0, 1), (2, 3), (4, 5)), ((0, 1), (2, 3), (4, 6)), ((0, 1), (2, 3), (5, 6)), ((0, 1), (2, 4), (3, 5)), ((0, 1), (2, 4), (3, 6)), ((0, 1), (2, 4), (5, 6)), ((0, 1), (2, 5), (3, 4)), ((0, 1), (2, 5), (3, 6)), ((0, 1), (2, 5), (4, 6)), ((0, 1), (2, 6), (3, 4)), ((0, 1), (2, 6), (3, 5)), ((0, 1), (2, 6), (4, 5)), ((0, 1), (3, 4), (5, 6)), ((0, 1), (3, 5), (4, 6)), ((0, 1), (3, 6), (4, 5)), ((0, 2), (1, 3), (4, 5)), ((0, 2), (1, 3), (4, 6)), ((0, 2), (1, 3), (5, 6)), ((0, 2), (1, 4), (3, 5)), ((0, 2), (1, 4), (3, 6)), ((0, 2), (1, 4), (5, 6)), ((0, 2), (1, 5), (3, 4)), ((0, 2), (1, 5), (3, 6)), ((0, 2), (1, 5), (4, 6)), ((0, 2), (1, 6), (3, 4)), ((0, 2), (1, 6), (3, 5)), ((0, 2), (1, 6), (4, 5)), ((0, 2), (3, 4), (5, 6)), ((0, 2), (3, 5), (4, 6)), ((0, 2), (3, 6), (4, 5)), ((0, 3), (1, 2), (4, 5)), ((0, 3), (1, 2), (4, 6)), ((0, 3), (1, 2), (5, 6)), ((0, 3

In [1]:
import numpy as np
from Solver import *


rng = np.random.default_rng(4)
logger = Logger()
timer = Timer(100)

timer.start_timer()

input_data = InputData("Testinstanzen/testing.json", consider_minimum_distance=False)

nhood_1block = K_BlockNeighborhood(input_data, rng, logger, timer, "1-Block")
nhood_1insertion = K_InsertionNeighborhood(input_data, rng, logger, timer, "1-Insertion")

permutation = np.array(range(input_data.n))

print(nhood_1block.combinations)
combination_1block = (1, 6) 
combination_1insertion = ((1,6),) # takes a input structure made for multiple operations after each other

move_1block = nhood_1block._make_move(permutation, combination_1block)
move_1insertion = nhood_1insertion._make_move(permutation, combination_1insertion)


print("1-Block Neighborhood")
print("-"*100)
print(f"Combinations: {nhood_1block.combinations}")
print(f"Move: {combination_1block}")
print(f"Initial permutation vs. 1-block permutation:")
print(permutation)
print(move_1block.permutation)  
print("-"*100)
print("-"*100)
print("1-Insertion Neighborhood")
print("-"*100)
print(f"Combinations: {nhood_1insertion.combinations}")
print(f"Move: {combination_1insertion}")
print("Initial permutation vs. 1-insertion permutation:")
print(permutation)
print(move_1insertion.permutation)
print("-"*100)
print("-"*100)


[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (3, 0), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (4, 0), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
1-Block Neighborhood
----------------------------------------------------------------------------------------------------
Combinations: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (3, 0), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (4, 0), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5)]
Move: (1, 6)
Initial permutation vs. 1-block permutation:
[0 1 2 3 4 5 6]
[0 2 3 4 5 6 1]
----------------------------------------------------------------------------------------------------

# Running the Solver

Parameter, die Ausgewählt werden können:
- neighborhood_evaluation_strategy: BestImprovement, FirstImprovement
- neighborhood_types: k-Swap, k-Insertion, k-Cut, k-Permute, k-Block
- search_procedure: LocalSearch, B-VND, C-VND, P-VND, U-VND, Skip
- neighborhood_change_procedure: Sequential, Cyclic, Pipe
- alpha_neighborhood_change: some value > 0 für skewed VNS

- constructive_method: k-increasing_dof, ordered_greedy, dynamic_ordered_greedy, k-random

Wichtige Punkte zum Testen:
1. Es ist unmöglich alle zu testen, deshalb wird sich auf ein paar konzentriert. 
2. Ich versuche meistens die Nachbarschaften nach Größe zu ordnen (stimmt nicht immer genau). 
3. Es werden nur sequenzielle Neighborhood Changes getestes (in neighborhood_change_procedure und B-VND). 
4. Es wird ausschließlich mit increasing_dof gestartet. 
5. Skewed VNS wird gar nicht getestet.
6. Es wird immer die beste Lösung gespeichert.
7. Pro Konfiguration wird die relative Abweichung zu der bereitgestellten Lösung ermittelt und summiert für alle Testinstanzen mit und ohne Berücksichtung der Matrix M
8. Alle Konfigurationen sind darauf ausgelegt bis n=25 gut zu laufen, für größere n könnte die Initialisierung oder die Evaluation zu lange dauern

In [4]:
from Solver import Solver
import pandas as pd

def run_solver(path, consider_minimum_distance, vns_params, const_params, time_limit=180):
    print("-"*100)
    print(f"Running solver for {path} with consider_minimum_distance={consider_minimum_distance}")
    solver = Solver(path, seed=42, consider_minimum_distance=consider_minimum_distance, constructive_params=const_params, vns_params=vns_params, time_limit=time_limit)
    solver.run_search()
    print("\n")
    rel_diff = solver.solution.check_optimal_solution(solver.input_data, print_diff=True)
    return (solver, rel_diff)

def analyze_solution_history(solution_history):
    # Convert list of dicts to DataFrame
    df = pd.DataFrame(solution_history)
    
    # Count occurrences of each neighborhood type
    search_counts = df['search_neighborhood'].value_counts()
    shaking_counts = df['shaking_neighborhood'].value_counts()
    
    print("\nSearch Neighborhood Counts:")
    print(search_counts)
    print("\nShaking Neighborhood Counts:") 
    print(shaking_counts)
    
    # Get time of last solution
    last_time = df['time'].iloc[-1]
    print(f"\nTime of last solution: {last_time:.2f} seconds")


def run_all_paths(const_params, vns_params):
    paths = ["Testinstanzen/scr12.json", "Testinstanzen/scr15.json", "Testinstanzen/scr20.json", "Testinstanzen/tai25b.json"]

    rel_diff = 0
    for path in paths:
        solver_false, rel_diff_false = run_solver(path, False, vns_params, const_params=const_params)
        analyze_solution_history(solver_false.solution_history)
        solver_false.save_solution_and_logs()

        solver_true, rel_diff_true = run_solver(path, True, vns_params, const_params=const_params)
        analyze_solution_history(solver_true.solution_history)
        solver_true.save_solution_and_logs()

        rel_diff += rel_diff_false + rel_diff_true
    
    print("-"*100)
    print(f"Total relative difference: {rel_diff}")

ImportError: dlopen(/opt/miniconda3/lib/python3.12/site-packages/numpy/core/_multiarray_umath.cpython-312-darwin.so, 0x0002): Library not loaded: @loader_path/libgfortran.5.dylib
  Referenced from: <7048D1FD-B9A9-3F6A-B752-AE8BE69F6C3F> /opt/miniconda3/lib/python3.12/site-packages/numpy/.dylibs/libopenblas64_.0.dylib
  Reason: tried: '/opt/miniconda3/lib/python3.12/site-packages/numpy/.dylibs/libgfortran.5.dylib' (no such file), '/usr/local/lib/libgfortran.5.dylib' (no such file), '/usr/lib/libgfortran.5.dylib' (no such file, not in dyld cache)

ImportError: dlopen(/opt/miniconda3/lib/python3.12/site-packages/numpy/core/_multiarray_umath.cpython-312-darwin.so, 0x0002): Library not loaded: @loader_path/libgfortran.5.dylib
  Referenced from: <7048D1FD-B9A9-3F6A-B752-AE8BE69F6C3F> /opt/miniconda3/lib/python3.12/site-packages/numpy/.dylibs/libopenblas64_.0.dylib
  Reason: tried: '/opt/miniconda3/lib/python3.12/site-packages/numpy/.dylibs/libgfortran.5.dylib' (no such file), '/usr/local/lib/libgfortran.5.dylib' (no such file), '/usr/lib/libgfortran.5.dylib' (no such file, not in dyld cache)

## Base VNS mit unterschiedlichen Nachbarschaftskombinationen

In [5]:
constructive_params = {
    "constructive_method": "1-increasing_dof"
}

nhoods_1 = ["1-Swap", "1-Insertion", "2-Block", "3-Block", "2-Cut", "2-Swap", "3-Permute", "3-Cut"]

nhoods_2 = ["2-Permute", "3-Permute", "4-Permute", "2-Swap"]


vns_params1 = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "LocalSearch",
    "neighborhood_types": nhoods_1,
    "neighborhood_change_procedure": "Sequential",
}

vns_params2 = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "LocalSearch",
    "neighborhood_types": nhoods_2,
    "neighborhood_change_procedure": "Sequential",
}

In [8]:
run_all_paths(constructive_params, vns_params1)

----------------------------------------------------------------------------------------------------
Running solver for Testinstanzen/scr12.json with consider_minimum_distance=False
Starting search.
Starting construction phase.
Generating an initial solution according to 1-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [4, 7, 9, 6, 11, 10, 2, 3, 8, 1, 0, 5] results in a cost of 36884
Starting variable neighborhood search phase.
New solution: The permutation [4, 6, 9, 2, 10, 3, 7, 11, 8, 5, 0, 1] results in a cost of 32490
New solution: The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410
VNS finished after 1550 iterations.
Best found Solution.
The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410


Optimal solution: 31410, Solution: 31410
Relative difference in %: 0.00%

Search Neighborhood Counts:
search_neighborhood
None      1
1-Swap    1
2-Swap    1
Name: count, dtype: int

👉 bei den kleineren Testinstanzen findet es nicht die optimalen lösungen

👉 block- und cut-Moves helfen nie und insertion selten, permutation auch nur einmal (cuts und blocks sind für das Problem eher unpassend, es bringt anscheinend nichts Standorte gemeinsam zu bewegen -> Fokus auf Swap, Insertion und Permute Nachbarschaften später)

👉 viele Iterationen geschaft, da größere Nachbarschaften nicht ausgewählt (würde auch bei Base DNS zu oft drankommen und die Suche behindern) 


In [18]:
run_all_paths(constructive_params, vns_params2)

----------------------------------------------------------------------------------------------------
Running solver for Testinstanzen/scr12.json with consider_minimum_distance=False
Starting search.
Starting construction phase.
Generating an initial solution according to 1-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [4, 7, 9, 6, 11, 10, 2, 3, 8, 1, 0, 5] results in a cost of 36884
Starting variable neighborhood search phase.
New solution: The permutation [4, 6, 9, 2, 10, 3, 7, 11, 8, 5, 0, 1] results in a cost of 32490
New solution: The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410
VNS finished after 612 iterations.
Best found Solution.
The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410


Optimal solution: 31410, Solution: 31410
Relative difference in %: 0.00%

Search Neighborhood Counts:
search_neighborhood
None         1
2-Permute    1
4-Permute    1
Name: count, dt

👉 schneidet am anfang besser ab als Konfiguration vorher

👉 aber bei den großen Testinstanzen nicht ganz so gut

👉  schaft nicht so viele iteration wegen 4-Permute

## FirstImprovement Base VNS

In [10]:
vns_params_first = {
    "neighborhood_evaluation_strategy": "FirstImprovement",
    "search_procedure": "LocalSearch",
    "neighborhood_types": ["1-Swap", "1-Insertion", "3-Permute", "2-Swap", "4-Permute", "3-Swap"],
    "neighborhood_change_procedure": "Sequential",
}

run_all_paths(constructive_params, vns_params_first)

----------------------------------------------------------------------------------------------------
Running solver for Testinstanzen/scr12.json with consider_minimum_distance=False
Starting search.
Starting construction phase.
Generating an initial solution according to 1-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [4, 7, 9, 6, 11, 10, 2, 3, 8, 1, 0, 5] results in a cost of 36884
Starting variable neighborhood search phase.
New solution: The permutation [4, 6, 5, 10, 2, 11, 7, 3, 1, 9, 0, 8] results in a cost of 32964
New solution: The permutation [9, 6, 5, 2, 10, 3, 7, 11, 1, 4, 8, 0] results in a cost of 31884
New solution: The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410
VNS finished after 548 iterations.
Best found Solution.
The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410


Optimal solution: 31410, Solution: 31410
Relative difference in %: 0.00%

Search Neigh

👉 ersten 4 perfekt, alle unter einem Prozent außer zwei, Großteil der Abweichung durch taib25b mit Constraint

👉 wegen 3 swap und 4 permute nicht so viele iterationen möglich (Steuern aber gute Shakes bei)

👉 Base VNS mit BestImprovement ist nicht so gut weil ich zu wenige kleine neighborhoods habe, und die großen führen zu weniger Iterationen 


## General VNS mit VND

- Nachbarschaften fürs Shaken können größer sein, müssen nur in angemessener Zeit initialisiert werden

### General VNS Konfig 1

In [11]:
vns_params_with_vnd = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "B-VND",
    "neighborhood_types": ["1-Swap", "1-Insertion", "3-Permute", "2-Swap", "4-Permute", "3-Swap"],
    "neighborhood_change_procedure": "Sequential",
    "vnd_neighborhood_types": ["1-Swap", "1-Insertion", "2-Swap"]
}

run_all_paths(constructive_params, vns_params_with_vnd)

----------------------------------------------------------------------------------------------------
Running solver for Testinstanzen/scr12.json with consider_minimum_distance=False
Starting search.
Starting construction phase.
Generating an initial solution according to 1-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [4, 7, 9, 6, 11, 10, 2, 3, 8, 1, 0, 5] results in a cost of 36884
Starting variable neighborhood search phase.
New solution: The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410
Time is up. B-VND procedure stopped.
VNS finished after 1619 iterations.
Best found Solution.
The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410


Optimal solution: 31410, Solution: 31410
Relative difference in %: 0.00%

Search Neighborhood Counts:
search_neighborhood
None      1
2-Swap    1
Name: count, dtype: int64

Shaking Neighborhood Counts:
shaking_neighborhood
None      1
1-Swa

👉 bisschen schlechter als FirstImprovement (liegt nur am letzten)

👉 bei allen anderen unter ein Prozent von der vorgegeben Lösung entfernt

👉 bei scr20 wird jeweils schon nach ein paar Sekunden die letzte Lösung gefunden 

👉 1-insertion kommt nicht so häufig in search neighborhood vor

### General VNS Konfig 2
- ohne insertion, dafür große permute
- initialisierung dauert länger aber häufiger shaken, um vllt nochmal weitentferntere lösungen auszuprobieren

In [12]:
vns_params_lean = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "B-VND",
    "neighborhood_types": ["1-Swap", "2-Swap", "3-Permute", "3-Swap", "4-Permute", "5-Permute"],
    "neighborhood_change_procedure": "Sequential",
    "vnd_neighborhood_types": ["1-Swap", "2-Swap"]
}

run_all_paths(constructive_params, vns_params_lean)



----------------------------------------------------------------------------------------------------
Running solver for Testinstanzen/scr12.json with consider_minimum_distance=False
Starting search.
Starting construction phase.
Generating an initial solution according to 1-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [4, 7, 9, 6, 11, 10, 2, 3, 8, 1, 0, 5] results in a cost of 36884
Starting variable neighborhood search phase.
New solution: The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410
Time is up.
Time is up. B-VND procedure stopped.
VNS finished after 2034 iterations.
Best found Solution.
The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410


Optimal solution: 31410, Solution: 31410
Relative difference in %: 0.00%

Search Neighborhood Counts:
search_neighborhood
None      1
2-Swap    1
Name: count, dtype: int64

Shaking Neighborhood Counts:
shaking_neighborhood
None 

👉 5-Permute hat direkt geliefert um 0 Prozent Abweichung bei scr20 ohne Mindestdistanzen zu treffen

👉 letzte aber wieder schlechter und versaut den Schnitt

### General VNS Konfig 3
- große shakes und kleines searches
- 1 swap, 2-swap 3-Permute sind eh in 4-Permute und 5-Permute enthalten

In [16]:
vns_params_lean = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "B-VND",
    "neighborhood_types": ["3-Swap", "4-Permute", "5-Permute"],
    "neighborhood_change_procedure": "Sequential",
    "vnd_neighborhood_types": ["1-Swap", "2-Swap"]
}

run_all_paths(constructive_params, vns_params_lean)

----------------------------------------------------------------------------------------------------
Running solver for Testinstanzen/scr12.json with consider_minimum_distance=False
Starting search.
Starting construction phase.
Generating an initial solution according to 1-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [4, 7, 9, 6, 11, 10, 2, 3, 8, 1, 0, 5] results in a cost of 36884
Starting variable neighborhood search phase.
New solution: The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410
VNS finished after 1797 iterations.
Best found Solution.
The permutation [4, 6, 1, 2, 10, 3, 7, 11, 0, 5, 8, 9] results in a cost of 31410


Optimal solution: 31410, Solution: 31410
Relative difference in %: 0.00%

Search Neighborhood Counts:
search_neighborhood
None      1
2-Swap    1
Name: count, dtype: int64

Shaking Neighborhood Counts:
shaking_neighborhood
None      1
3-Swap    1
Name: count, dtype: int64

Tim

👉 keinen Fortschritt

### Reduced VNS

In [13]:
vns_params_lean = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "Skip",
    "neighborhood_types": ["1-Swap", "1-Insertion", "3-Permute", "2-Swap", "4-Permute", "3-Swap", "2-Insertion", "4-Permute"],
    "neighborhood_change_procedure": "Sequential",
}

run_all_paths(constructive_params, vns_params_lean)


----------------------------------------------------------------------------------------------------
Running solver for Testinstanzen/scr12.json with consider_minimum_distance=False
Starting search.
Starting construction phase.
Generating an initial solution according to 1-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [4, 7, 9, 6, 11, 10, 2, 3, 8, 1, 0, 5] results in a cost of 36884
Starting variable neighborhood search phase.
New solution: The permutation [4, 7, 9, 6, 11, 10, 2, 3, 8, 5, 0, 1] results in a cost of 35680
New solution: The permutation [1, 2, 9, 6, 11, 10, 7, 3, 8, 5, 0, 4] results in a cost of 34372
New solution: The permutation [1, 2, 9, 6, 3, 10, 7, 11, 8, 5, 0, 4] results in a cost of 34060
New solution: The permutation [1, 3, 9, 2, 10, 6, 7, 11, 8, 5, 0, 4] results in a cost of 33882
New solution: The permutation [1, 6, 9, 2, 10, 3, 7, 11, 8, 5, 0, 4] results in a cost of 33230
New solution: The permutation [8, 

KeyboardInterrupt: 

👉 keine Suche hat, wie zu erwarten, schon bei den ersten Testinstanzen Probleme die optimale Lösung zu finden

## Andere Eröffnungskonfiguration

In [ ]:
from Solver import *

def run_increasing_dof(path, consider_minimum_distance):
    rng = np.random.default_rng(42)
    timer = Timer(180)
    logger = Logger()
    inputData = InputData(path, consider_minimum_distance)
    constructive_heuristic = ConstructiveHeuristic(inputData, rng, timer, logger)


    timer.start_timer()
    # set seed back for comparability
    constructive_heuristic.rng = np.random.default_rng(4)
    solution_dof_50 = constructive_heuristic.increasing_dof(1_000_000)
    solution_dof_50.check_optimal_solution(inputData, print_diff=True)

test_cases = [(path, consider_min) 
         for path in ["Testinstanzen/scr12.json", "Testinstanzen/scr15.json", "Testinstanzen/scr20.json", "Testinstanzen/tai25b.json"]
         for consider_min in [True, False]]

for test_case in test_cases:
    run_increasing_dof(test_case[0], test_case[1])

Time is up at iteration 14338 in increasing degree of freedom greedy constructive heuristic.
Optimal solution: 33996, Solution: 33996
Relative difference in %: 0.00%
Time is up at iteration 19886 in increasing degree of freedom greedy constructive heuristic.
Optimal solution: 31410, Solution: 31410
Relative difference in %: 0.00%
Time is up at iteration 7509 in increasing degree of freedom greedy constructive heuristic.
Optimal solution: 61032, Solution: 61406
Relative difference in %: 0.61%
Time is up at iteration 10001 in increasing degree of freedom greedy constructive heuristic.
Optimal solution: 51140, Solution: 52968
Relative difference in %: 3.57%
Time is up at iteration 3104 in increasing degree of freedom greedy constructive heuristic.
Optimal solution: 117926, Solution: 122916
Relative difference in %: 4.23%
Time is up at iteration 4592 in increasing degree of freedom greedy constructive heuristic.
Optimal solution: 110030, Solution: 114844
Relative difference in %: 4.38%
Tim

👉 increasing_dof ist besser auf 3 min in der relativen Abweichung, als alle VNS Konfigurationen, die ich getestet habe

👉 schafft es aber nicht so gut die "optimale" Lösung zu treffen

👉 Kombination aus sehr vielen increasing_dof Lösungen und VNS

👉 500 mal ausführen dauert für n=25 ungefähr 50s 

👉 lieber die ersten General Vns Konfig als die Base VNS Konfig mit Firstimprovement testen, weil sie mehr Iterationen schafft und schneller gute Lösungen findet (weniger Zeit durch Eröffnungskonfig)

In [17]:
constructive_params_500_dof = {
    "constructive_method": "500-increasing_dof"
}

vns_params_with_vnd = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "B-VND",
    "neighborhood_types": ["1-Swap", "1-Insertion", "3-Permute", "2-Swap", "4-Permute", "3-Swap"],
    "neighborhood_change_procedure": "Sequential",
    "vnd_neighborhood_types": ["1-Swap", "1-Insertion", "2-Swap"]
}

run_all_paths(constructive_params_500_dof, vns_params_with_vnd)


----------------------------------------------------------------------------------------------------
Running solver for Testinstanzen/scr12.json with consider_minimum_distance=False
Starting search.
Starting construction phase.
Generating an initial solution according to 500-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [10, 5, 6, 1, 9, 0, 4, 8, 2, 7, 11, 3] results in a cost of 31884
Starting variable neighborhood search phase.
New solution: The permutation [2, 5, 10, 9, 1, 8, 4, 0, 11, 6, 3, 7] results in a cost of 31410
Time is up.
Time is up. B-VND procedure stopped.
VNS finished after 1805 iterations.
Best found Solution.
The permutation [2, 5, 10, 9, 1, 8, 4, 0, 11, 6, 3, 7] results in a cost of 31410


Optimal solution: 31410, Solution: 31410
Relative difference in %: 0.00%

Search Neighborhood Counts:
search_neighborhood
None      1
2-Swap    1
Name: count, dtype: int64

Shaking Neighborhood Counts:
shaking_neighborhood
Non

👉 klarer Sieger

👉 deutlich besser als alle anderen Konfigurationen

👉 höchste Abweichung sind 0,88 Prozent, zwei mal sogar negativ, 4 mal 0 Prozent und einmal ganz knapp an der 0 (0,08)

👉 zwei Faktoren für die guten Ergebnisse: Effizienz ist sehr wichtig (mehr Iterationen und mehr Exploration möglich), increasing_dof ist eine sehr starke Eröffnungsheuristik

### Lucky hits

Ein paar einzelne Ausführungen, die zu sehr guten Lösungen geführt haben

In [ ]:
construction_params = {
    "constructive_method": "10-increasing_dof",
}

vns_params = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "B-VND",
    "neighborhood_types": ["1-Swap", "1-Insertion", "2-Block", "3-Block", "2-Cut", "2-Swap", "3-Permute", "3-Cut", "2-Insertion", "3-Swap", "4-Permute", "4-Swap"],
    "neighborhood_change_procedure": "Sequential",
    "vnd_neighborhood_types": ["1-Swap", "1-Insertion", "2-Swap"]
}

solver = Solver("Testinstanzen/scr20.json", seed=42, consider_minimum_distance=False, constructive_params=construction_params, vns_params=vns_params, time_limit=180)

solver.run_search()

solver.solution.check_optimal_solution(solver.input_data, print_diff=True)
solver.save_solution_and_logs()
analyze_solution_history(solver.solution_history)

solver = Solver("Testinstanzen/scr20.json", seed=42, consider_minimum_distance=True, constructive_params=construction_params, vns_params=vns_params, time_limit=180)

solver.run_search()

solver.solution.check_optimal_solution(solver.input_data, print_diff=True)
solver.save_solution_and_logs()
analyze_solution_history(solver.solution_history)



Starting search.
Starting construction phase.
Generating an initial solution according to 10-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [8, 10, 17, 6, 15, 19, 11, 3, 14, 9, 4, 5, 12, 13, 16, 0, 1, 18, 2, 7] results in a cost of 131208
Starting variable neighborhood search phase.
New solution: The permutation [16, 15, 10, 14, 19, 18, 11, 7, 8, 9, 1, 5, 12, 13, 17, 0, 4, 2, 6, 3] results in a cost of 115072
New solution: The permutation [1, 15, 6, 14, 2, 19, 11, 7, 12, 5, 16, 9, 0, 4, 8, 17, 13, 18, 10, 3] results in a cost of 114440
New solution: The permutation [1, 11, 18, 15, 3, 19, 7, 6, 16, 13, 0, 12, 5, 17, 9, 4, 8, 10, 14, 2] results in a cost of 113490
New solution: The permutation [1, 11, 10, 15, 3, 19, 7, 6, 17, 13, 0, 12, 5, 9, 8, 4, 16, 18, 14, 2] results in a cost of 112648
New solution: The permutation [1, 11, 10, 15, 3, 19, 7, 6, 8, 9, 17, 13, 0, 5, 4, 16, 12, 18, 14, 2] results in a cost of 110994
New solution: The

In [43]:
construction_params = {
    "constructive_method": "10-increasing_dof",
    "disable_special_order": True 
}

vns_params = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "B-VND",
    "neighborhood_types": ["1-Swap", "1-Insertion", "2-Block", "3-Block", "2-Cut", "2-Swap", "3-Permute", "3-Cut", "2-Insertion", "3-Swap", "4-Permute", "4-Swap"],
    "neighborhood_change_procedure": "Sequential",
    "vnd_neighborhood_types": ["1-Swap", "1-Insertion", "2-Swap"]
}


solver = Solver("Testinstanzen/scr20.json", seed=42, consider_minimum_distance=True, constructive_params=construction_params, vns_params=vns_params, time_limit=180)

solver.run_search()

solver.solution.check_optimal_solution(solver.input_data, print_diff=True)
solver.save_solution_and_logs()
analyze_solution_history(solver.solution_history)

Starting search.
Starting construction phase.
Generating an initial solution according to 10-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [12, 11, 19, 6, 15, 14, 2, 3, 17, 13, 4, 9, 8, 16, 5, 0, 1, 18, 10, 7] results in a cost of 146814
Starting variable neighborhood search phase.
New solution: The permutation [12, 11, 18, 10, 3, 15, 7, 6, 16, 13, 0, 9, 5, 17, 1, 4, 8, 19, 14, 2] results in a cost of 119068
New solution: The permutation [16, 15, 18, 14, 6, 19, 11, 7, 12, 13, 0, 9, 5, 17, 1, 4, 8, 2, 10, 3] results in a cost of 118730
New solution: The permutation [3, 8, 5, 4, 16, 0, 12, 13, 7, 10, 18, 11, 2, 6, 19, 14, 15, 1, 9, 17] results in a cost of 116872
VNS finished after 69 iterations.
Best found Solution.
The permutation [3, 8, 5, 4, 16, 0, 12, 13, 7, 10, 18, 11, 2, 6, 19, 14, 15, 1, 9, 17] results in a cost of 116872
Optimal solution: 117926, Solution: 116872
Relative difference in %: -0.89%
Saving solution and logs to f

In [42]:
from Solver import Solver

construction_params = {
    "constructive_method": "10-increasing_dof"
}

vns_params = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "B-VND",
    "neighborhood_types": ["1-Swap", "1-Insertion", "3-Permute", "2-Swap", "4-Permute", "3-Swap"],
    "neighborhood_change_procedure": "Sequential",
    "vnd_neighborhood_types": ["1-Swap", "2-Swap"]
}


solver = Solver("Testinstanzen/tai25b.json", seed=4, consider_minimum_distance=False, constructive_params=construction_params, vns_params=vns_params, time_limit=180)

solver.run_search()

solver.solution.check_optimal_solution(solver.input_data, print_diff=True)
solver.save_solution_and_logs()
analyze_solution_history(solver.solution_history)

solver = Solver("Testinstanzen/tai25b.json", seed=4, consider_minimum_distance=True, constructive_params=construction_params, vns_params=vns_params, time_limit=180)

solver.run_search()

solver.solution.check_optimal_solution(solver.input_data, print_diff=True)
solver.save_solution_and_logs()
analyze_solution_history(solver.solution_history)

Starting search.
Starting construction phase.
Generating an initial solution according to 10-increasing_dof.
Solution updated in constructive phase.
Constructive solution found.
The permutation [17, 13, 3, 8, 6, 23, 16, 4, 12, 0, 14, 20, 11, 2, 10, 19, 1, 18, 7, 24, 21, 15, 22, 9, 5] results in a cost of 402669340
Starting variable neighborhood search phase.
New solution: The permutation [17, 3, 0, 8, 12, 11, 13, 4, 6, 23, 14, 20, 16, 2, 10, 7, 21, 22, 1, 9, 15, 24, 18, 5, 19] results in a cost of 378915093
New solution: The permutation [3, 24, 15, 8, 12, 4, 5, 23, 6, 16, 9, 2, 14, 19, 17, 1, 21, 22, 7, 11, 20, 10, 13, 0, 18] results in a cost of 346082183
New solution: The permutation [3, 9, 5, 12, 6, 17, 23, 14, 8, 16, 24, 15, 4, 19, 2, 1, 21, 22, 7, 11, 20, 10, 13, 0, 18] results in a cost of 345475531
Time is up. B-VND procedure stopped.
VNS finished after 86 iterations.
Best found Solution.
The permutation [3, 9, 5, 12, 6, 17, 23, 14, 8, 16, 24, 15, 4, 19, 2, 1, 21, 22, 7, 11, 20,

# Mögliche Verbesserungen

- jede Nachbarschaft implementiert eine Delta-Kostenfunktion um dieses Bottleneck schneller zu machen

- Concurrency einbauen für BestImprovement, Berechnung auf CPU-Kerne verteilen 

- Input strenger validieren (unzulässige Parameterkonfigurationen werfen keinen Error)

# Überprüfung

<mark> wenn n größer ist als 25, können Fehler auftreten <mark>
- Eröffnungsverfahren könnte länger als 3 min dauern
- die Initialisierung von 4-Permute und 3-Swap könnte Memory- und Zeitlimits sprengen
- wenige Iterationen könnten durchgeführt werden durch große Neighborhoods

In [ ]:
from Solver import Solver

path = "Testinstanzen/tai25b.json"
minimum_distance = True

time_limit = 180
seed = 42   


constructive_params = {
    "constructive_method": "500-increasing_dof"
}

vns_params_with_vnd = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "B-VND",
    "neighborhood_types": ["1-Swap", "1-Insertion", "3-Permute", "2-Swap", "4-Permute", "3-Swap"],
    "neighborhood_change_procedure": "Sequential",
    "vnd_neighborhood_types": ["1-Swap", "1-Insertion", "2-Swap"]
}



solver = Solver(path, seed=seed, consider_minimum_distance=minimum_distance, constructive_params=constructive_params, vns_params=vns_params_with_vnd, time_limit=time_limit)

solver.run_search()

solver.save_solution_and_logs()

#### Backup falls n größer als 25 ist und die Konfig oben fehlschlägt

In [ ]:
from Solver import Solver

path = "Testinstanzen/tai25b.json"
minimum_distance = True

time_limit = 180
seed = 42   

constructive_params = {
    "constructive_method": "1-increasing_dof"
}

vns_params = {
    "neighborhood_evaluation_strategy": "BestImprovement",
    "search_procedure": "LocalSearch",
    "neighborhood_types": ["1-Swap", "1-Insertion", "2-Block", "3-Block", "2-Cut", "2-Swap"],
    "neighborhood_change_procedure": "Sequential",
}


solver = Solver(path, seed=seed, consider_minimum_distance=minimum_distance, constructive_params=constructive_params, vns_params=vns_params_with_vnd, time_limit=time_limit)

solver.run_search()

solver.save_solution_and_logs()